# Weather Play Golf Dataset

## Show the Dataset

In [ ]:
import pandas as pd, numpy as np

In [3]:
df = pd.read_csv('weather_nominal.csv')

In [4]:
df

,Outlook,Temperature,Humidity,Windy,Play golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes
5,Sunny,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Rainy,Mild,High,False,No
8,Rainy,Cool,Normal,False,Yes
9,Sunny,Mild,Normal,False,Yes


The above dataset shows 14 rows. Each row has 5 columns where the first 4 columns are the attributes that may or may not contribute to the decision whether to play golf or not indicated in the last column. We would like to make use of the 4 attributes in the future dataset to predict the decision. Sometime the prediciton is also called classificaiton.

## Describe the Dataset

In [5]:
df.describe()

,Outlook,Temperature,Humidity,Windy,Play golf
count,14,14,14,14,14
unique,3,3,2,2,2
top,Rainy,Mild,Normal,False,Yes
freq,5,6,7,8,9


Trying to describe the dataset gives us some general ideas. 

The description of the dataset shows Outlook and Temperature are both 3-class attributes as there are only 3 possibilities of (Sunny, Rainy, Overcast) and (Hot, Mild, Cool), and Humidity and Windy are both 2-class attribute as there are only 2 possibilities of (Normal, High) and (True, False).

The other description is that the last column Play golf has the top possibility of Yes with frequency of 9 out of 14. If we make prediction purely using this column, we may say there is a 9/14 = 64% of chance that the decision is Yes.

SyntaxError: invalid syntax (<ipython-input-7-81762121b108>, line 1)